All instructions are provided for R. I am going to reproduce them in Python as best as I can.

# Preface

In this problem we will investigate the t-statistic for the null hypothesis $H_0 : \beta = 0$ in simple linear regression without an intercept. To begin, we generate a predictor `x` and a response `y` as follows.
```
> set.seed(1)
> x = rnorm(100)
> y = 2*x + rnorm(100)
```

In [1]:
import numpy as np
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
np.random.seed(1)
x = np.random.normal(size=100)
y = 2*x + np.random.normal(size=100)

# (a)

From the textbook, p. 123:
> Perform a simple linear regression of `y` onto `x` , without an intercept. Report the coefficient estimate $\hat{\beta}$, the standard error of this coefficient estimate, and the t-statistic and p-value associated with the null hypothesis $H_0: \beta = 0$. Comment on these results. (You can perform regression without an intercept using the command `lm(y∼x+0)`.)

In [3]:
model_a = sm.regression.linear_model.OLS(y, x).fit(use_t=True)
print(f'x coefficient estimate: {model_a.params[0]:.3f}'
      , f'Standard error: {model_a.bse[0]:.3f}'
      , f't-statistic: {model_a.tvalues[0]:.3f}'
      , f'p-value: {model_a.pvalues[0]}'
      , sep='\n'
)

x coefficient estimate: 2.107
Standard error: 0.106
t-statistic: 19.792
p-value: 3.457375739189495e-36


$\hat{\beta_x} = 2.107$ is a least-squares estimate of $\beta_x$ in $y \approx \beta_x \cdot x$.

# (b)

From the textbook, p. 123:
> Now perform a simple linear regression of `x` onto `y` without an intercept, and report the coefficient estimate, its standard error, and the corresponding t-statistic and p-values associated with the null hypothesis $H_0: \beta = 0$. Comment on these results.

In [4]:
model_b = sm.regression.linear_model.OLS(x, y).fit(use_t=True)
print(f'y coefficient estimate: {model_b.params[0]:.3f}'
      , f'Standard error: {model_b.bse[0]:.3f}'
      , f't-statistic: {model_b.tvalues[0]:.3f}'
      , f'p-value: {model_b.pvalues[0]}'
      , sep='\n'
)

y coefficient estimate: 0.379
Standard error: 0.019
t-statistic: 19.792
p-value: 3.4573757391893976e-36


$\hat{\beta_y} = 0.379$ is a least-squares estimate of $\beta_y$ in $x \approx \beta_y \cdot y$.

# (c)

From the textbook, p. 123:
> What is the relationship between the results obtained in (a) and (b)?

The t-statistics and the p-values are equal; the coefficient estimates and the standard errors are not. I would expect $\hat{\beta_x} = 1 / \hat{\beta_y}$, but this is not what's happening ($2.107 \neq 2.639$).

# (d)

From the textbook, p. 123:
> For the regression of `y` onto `x` without an intercept, the t-statistic for $H_0: \beta = 0$ takes the form $\hat{\beta}/\text{SE}(\hat{\beta})$, where $\hat{\beta}$ is
$$
\hat{\beta} = \left( \sum_{i=1}^n x_i y_i \right) \Biggm/ \left( \sum_{i'=1}^n x_{i'}^2 \right),
$$
and where $$\text{SE}(\hat{\beta}) = \sqrt{\frac{\sum_{i=1}^n (y_i - x_i \hat{\beta})^2}{(n-1) \sum_{i'=1}^n x_{i'}^2}}$$ <br> Show  numerically in R, that the t-statistic can be written as
$$
  \frac{(\sqrt{n - 1}) \sum_{i=1}^n x_i y_i}
       {\sqrt{\left(
                \sum_{i=1}^n x_i^2
              \right)
              \left(
                \sum_{i=1}^n y_{i}^2
              \right)
              - \left(
                  \sum_{i=1}^n x_{i} y_{i}
                \right)^2
             }
       }
$$

In [5]:
n = len(x)
beta = np.dot(x, y) / np.sum(x**2)
se_beta = np.sqrt(np.sum((y - x*beta)**2)
                  / 
                  ((n - 1) * np.sum(x**2))
                 )
t1 = beta / se_beta

t2 = np.sqrt(n - 1) * np.dot(x, y) / np.sqrt(np.sum(x**2) * np.sum(y**2) - np.dot(x, y)**2)

print(t1)
print(t2)

19.7918019870912
19.791801987091205


Looks good to me.

# (e)

From the textbook, p. 124:
> Using the results from (d), argue that the t-statistic for the regression of `y` onto `x` is the same as the t-statistic for the regression of `x` onto `y`.

If you replace $x$ with $y$ and vice versa in 
$$
t=\frac{(\sqrt{n - 1}) \sum_{i=1}^n x_i y_i}
       {\sqrt{\left(
                \sum_{i=1}^n x_i^2
              \right)
              \left(
                \sum_{i=1}^n y_{i}^2
              \right)
              - \left(
                  \sum_{i=1}^n x_{i} y_{i}
                \right)^2
             }
       },
$$
the equation does not change.

# (f)

From the textbook, p. 124:
> In R, show that when regression is performed with an intercept, the t-statistic for $H_0: \beta_1 = 0$ is the same for the regression of `y` onto `x` as it is for the regression of `x` onto `y`.

In [6]:
x_intercept = np.ones((x.size, 2))
x_intercept[:, 0] = x
y_intercept = np.ones((y.size, 2))
y_intercept[:, 0] = y
model1 = sm.regression.linear_model.OLS(y, x_intercept).fit(use_t=True)
model2 = sm.regression.linear_model.OLS(x, y_intercept).fit(use_t=True)
print(f'y ~ x+intercept, t-statistic: {model1.tvalues[0]:.3f}\n'
      f'x ~ y+intercept, t_statistic: {model2.tvalues[0]:.3f}'
)

y ~ x+intercept, t-statistic: 19.783
x ~ y+intercept, t_statistic: 19.783
